In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

In [6]:
clk_18_20 = pd.read_csv( loc_ts + "\\clk_18_20.csv" );
clk_21_23 = pd.read_csv( loc_ts + "\\clk_21_23.csv" );
clk_24_26 = pd.read_csv( loc_ts + "\\clk_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
auc_18_20["date"] =  pd.to_datetime( auc_18_20["date"] ); auc_18_20["_st"] = auc_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_18_20["_st"] = auc_18_20["_st"].dt.total_seconds();
auc_21_23["date"] =  pd.to_datetime( auc_21_23["date"] ); auc_21_23["_st"] = auc_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_21_23["_st"] = auc_21_23["_st"].dt.total_seconds(); 
auc_24_26["date"] =  pd.to_datetime( auc_24_26["date"] ); auc_24_26["_st"] = auc_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_24_26["_st"] = auc_24_26["_st"].dt.total_seconds(); 

In [8]:
auc_18_20_first_auc = auc_18_20.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_21_23_first_auc = auc_21_23.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_24_26_first_auc = auc_24_26.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_auc"

In [10]:
rh_18_20 = auc_18_20_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = auc_21_23_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = auc_18_20_first_auc[ ["_st"] ]
tg_21_23 = auc_21_23_first_auc[ ["_st"] ]

## <span style="color:green"> **Cantidad de Clicks en la ventana anterior** </span> 

In [69]:
cant_clk_18_20 = rh_18_20.copy()
cant_clk_21_23 = rh_21_23.copy()

cant_clk_18_20 = cant_clk_18_20.merge( clk_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_clk_21_23 = cant_clk_21_23.merge( clk_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_clk_18_20.fillna( 0, inplace = True )
cant_clk_21_23.fillna( 0, inplace = True )

cant_clk_18_20.to_csv( loc_ftr + "\\cant_clk_18_20.csv", index = False )
cant_clk_21_23.to_csv( loc_ftr + "\\cant_clk_21_23.csv", index = False )

## <span style="color:green"> **Promedio del TimeToClick es un anuncio** </span> 

In [ ]:
ttc_mean_18_20 = rh_18_20.copy()
ttc_mean_21_23 = rh_21_23.copy()

ttc_by_rh_18_20 = clk_18_20[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_18_20 = ttc_by_rh_18_20.reset_index()
ttc_by_rh_21_23 = clk_21_23[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_21_23 = ttc_by_rh_21_23.reset_index()

ttc_mean_18_20 = ttc_mean_18_20.merge( ttc_by_rh_18_20, how = "left", on = "ref_hash" )
ttc_mean_21_23 = ttc_mean_21_23.merge( ttc_by_rh_21_23, how = "left", on = "ref_hash" )

Casi todos los valores nulos!



## <span style="color:green"> **Tiempo hasta el primer click** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [ ]:
frst_clk_18_20 = rh_18_20.copy()
frst_clk_21_23 = rh_21_23.copy()

first_click_rh_auc_18_20 = clk_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_click_rh_auc_21_23 = clk_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_click_rh_auc_18_20["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_click_rh_auc_21_23["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_click_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
first_click_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

frst_clk_18_20 = frst_clk_18_20.merge( first_click_rh_auc_18_20, how = "left", on = "ref_hash" )
frst_clk_21_23 = frst_clk_21_23.merge( first_click_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_clk_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_clk_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_clk_18_20.to_csv( loc_ftr + "\\frst_clk_18_20.csv", index = False )
frst_clk_21_23.to_csv( loc_ftr + "\\frst_clk_21_23.csv", index = False )

## <span style="color:green"> **Recibio clicks entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
clk_18_20['clk_16_20'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 21) 
clk_21_23['clk_16_20'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 21)

clk_after_18_20 = rh_18_20.copy()
clk_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_18_20 = clk_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_after_21_23 = clk_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_after_18_20["clk_16_20"] = (clk_after_18_20["clk_16_20"] > 1).astype('int8')
clk_after_21_23["clk_16_20"] = (clk_after_21_23["clk_16_20"] > 1).astype('int8')

clk_after_18_20['me_clk_16_20'] = ( clk_after_18_20.groupby('clk_16_20').transform('count') ) / len(clk_after_18_20['clk_16_20'])
clk_after_21_23['me_clk_16_20'] = ( clk_after_21_23.groupby('clk_16_20').transform('count') ) / len(clk_after_21_23['clk_16_20'])

del clk_after_18_20["clk_16_20"]
del clk_after_21_23["clk_16_20"]

clk_after_18_20.to_csv( loc_ftr + "\\clk_afme_18_20.csv", index = False )
clk_after_21_23.to_csv( loc_ftr + "\\clk_afme_21_23.csv", index = False )

del clk_18_20['clk_16_20']
del clk_21_23['clk_16_20']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [32]:
clk_18_20['clk_16_20'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 21) 
clk_21_23['clk_16_20'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 21)

clk_after_18_20 = rh_18_20.copy()
clk_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_18_20 = clk_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_after_21_23 = clk_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_after_18_20["clk_16_20"] = (clk_after_18_20["clk_16_20"] > 1).astype('int8')
clk_after_21_23["clk_16_20"] = (clk_after_21_23["clk_16_20"] > 1).astype('int8')

clk_after_18_20.to_csv( loc_ftr + "\\clk_afoh_18_20.csv", index = False )
clk_after_21_23.to_csv( loc_ftr + "\\clk_afoh_21_23.csv", index = False )

del clk_18_20['clk_16_20']
del clk_21_23['clk_16_20']

## <span style="color:green"> **Recibio clikcs entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [33]:
clk_18_20['clk_21_3'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 20)
clk_21_23['clk_21_3'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 20)

clk_night_18_20 = rh_18_20.copy()
clk_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_18_20 = clk_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_night_21_23 = clk_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_night_18_20["clk_21_3"] =  (clk_night_18_20["clk_21_3"] > 1).astype('int8')
clk_night_21_23["clk_21_3"] =  (clk_night_21_23["clk_21_3"] > 1).astype('int8')

clk_night_18_20['me_clk_21_3'] = ( clk_night_18_20.groupby('clk_21_3').transform('count') ) / len(clk_night_18_20['clk_21_3'])
clk_night_21_23['me_clk_21_3'] = ( clk_night_21_23.groupby('clk_21_3').transform('count') ) / len(clk_night_21_23['clk_21_3'])

del clk_night_18_20["clk_21_3"]
del clk_night_21_23["clk_21_3"]

clk_night_18_20.to_csv( loc_ftr + "\\clk_ngme_18_20.csv", index = False )
clk_night_21_23.to_csv( loc_ftr + "\\clk_ngme_21_23.csv", index = False )

del clk_18_20['clk_21_3']
del clk_21_23['clk_21_3']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [34]:
clk_18_20['clk_21_3'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 20)
clk_21_23['clk_21_3'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 20)

clk_night_18_20 = rh_18_20.copy()
clk_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_18_20 = clk_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_night_21_23 = clk_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_night_18_20["clk_21_3"] =  (clk_night_18_20["clk_21_3"] > 1).astype('int8')
clk_night_21_23["clk_21_3"] =  (clk_night_21_23["clk_21_3"] > 1).astype('int8')

clk_night_18_20.to_csv( loc_ftr + "\\clk_ngoh_18_20.csv", index = False )
clk_night_21_23.to_csv( loc_ftr + "\\clk_ngoh_21_23.csv", index = False )

del clk_18_20['clk_21_3']
del clk_21_23['clk_21_3']

## <span style="color:green"> **Recibio clicks entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [35]:
clk_18_20['clk_11_15'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 16) 
clk_21_23['clk_11_15'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 16)

clk_midday_18_20 = rh_18_20.copy()
clk_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_18_20 = clk_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_midday_21_23 = clk_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_midday_18_20["clk_11_15"] =  (clk_midday_18_20["clk_11_15"] > 1).astype('int8')
clk_midday_21_23["clk_11_15"] =  (clk_midday_21_23["clk_11_15"] > 1).astype('int8')

clk_midday_18_20['me_clk_11_15'] = ( clk_midday_18_20.groupby('clk_11_15').transform('count') ) / len(clk_midday_18_20['clk_11_15'])
clk_midday_21_23['me_clk_11_15'] = ( clk_midday_21_23.groupby('clk_11_15').transform('count') ) / len(clk_midday_21_23['clk_11_15'])

del clk_midday_18_20["clk_11_15"]
del clk_midday_21_23["clk_11_15"]

clk_midday_18_20.to_csv( loc_ftr + "\\clk_mdme_18_20.csv", index = False )
clk_midday_21_23.to_csv( loc_ftr + "\\clk_mdme_21_23.csv", index = False )

del clk_18_20['clk_11_15']
del clk_21_23['clk_11_15']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [36]:
clk_18_20['clk_11_15'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 16) 
clk_21_23['clk_11_15'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 16)

clk_midday_18_20 = rh_18_20.copy()
clk_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_18_20 = clk_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_midday_21_23 = clk_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_midday_18_20["clk_11_15"] =  (clk_midday_18_20["clk_11_15"] > 1).astype('int8')
clk_midday_21_23["clk_11_15"] =  (clk_midday_21_23["clk_11_15"] > 1).astype('int8')

clk_midday_18_20.to_csv( loc_ftr + "\\clk_mdoh_18_20.csv", index = False )
clk_midday_21_23.to_csv( loc_ftr + "\\clk_mdoh_21_23.csv", index = False )

del clk_18_20['clk_11_15']
del clk_21_23['clk_11_15']

## <span style="color:green"> **Recibio clicks entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [38]:
clk_18_20['clk_4_10'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 11) 
clk_21_23['clk_4_10'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 11)

clk_morn_18_20 = rh_18_20.copy()
clk_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_18_20 = clk_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_morn_21_23 = clk_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_morn_18_20["clk_4_10"] =  (clk_morn_18_20["clk_4_10"] > 1).astype('int8')
clk_morn_21_23["clk_4_10"] =  (clk_morn_21_23["clk_4_10"] > 1).astype('int8')

clk_morn_18_20['me_clk_4_10'] = ( clk_morn_18_20.groupby('clk_4_10').transform('count') ) / len(clk_morn_18_20['clk_4_10'])
clk_morn_21_23['me_clk_4_10'] = ( clk_morn_21_23.groupby('clk_4_10').transform('count') ) / len(clk_morn_21_23['clk_4_10'])

del clk_morn_18_20["clk_4_10"]
del clk_morn_21_23["clk_4_10"]

clk_morn_18_20.to_csv( loc_ftr + "\\clk_mrme_18_20.csv", index = False )
clk_morn_21_23.to_csv( loc_ftr + "\\clk_mrme_21_23.csv", index = False )

del clk_18_20['clk_4_10']
del clk_21_23['clk_4_10']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [39]:
clk_18_20['clk_4_10'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 11) 
clk_21_23['clk_4_10'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 11)

clk_morn_18_20 = rh_18_20.copy()
clk_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_18_20 = clk_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_morn_21_23 = clk_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_morn_18_20["ins_4_10"] =  (clk_morn_18_20["clk_4_10"] > 1).astype('int8')
clk_morn_21_23["ins_4_10"] =  (clk_morn_21_23["clk_4_10"] > 1).astype('int8')

clk_morn_18_20.to_csv( loc_ftr + "\\clk_mroh_18_20.csv", index = False )
clk_morn_21_23.to_csv( loc_ftr + "\\clk_mroh_21_23.csv", index = False )

del clk_18_20['clk_4_10']
del clk_21_23['clk_4_10']